Try and play around with how to refactor moving objects

In [1]:
import matplotlib.pylab as plt 
%matplotlib inline
import numpy as np
import rubin_sim.maf_proto as maf
import pandas as pd
import sqlite3
import healpy as hp
import copy

from rubin_sim.data import get_baseline
from os.path import basename

from rubin_sim.moving_objects import DirectObs, Orbits

In [2]:
# load up 100 nights
baseline_file = get_baseline()
run_name = basename(baseline_file).replace('.db', '')
con = sqlite3.connect(baseline_file)
# Dataframe is handy for some calcs
df = pd.read_sql("select * from observations where night < 100;", con)
# But mostly want numpy array for speed.
visits_array = df.to_records(index=False)
con.close()

In [3]:
d_obs = DirectObs()

# Need to update this to "band" everywhere. Probably just set 
# DirectObs to run .read_filters with 'ugrizy' on __init__
# by default
filterlist = np.unique(visits_array["band"])
d_obs.read_filters(filterlist=filterlist)

In [4]:
positions_file = "/Users/yoachim/rubin_sim_data/orbits_precompute/trojan_5k.npz"
orbit_file = "/Users/yoachim/rubin_sim_data/orbits/trojan_5k.txt"

position_data = np.load(positions_file)
object_positions = position_data["positions"].copy()
object_mjds = position_data["mjds"].copy()
position_data.close()

In [5]:
# read in orbits
orbits = Orbits()
orbits.read_orbits(orbit_file)

In [6]:
object_observations = d_obs.run(
        orbits,
        visits_array,
        object_positions=object_positions,
        object_mjds=object_mjds,
    )

In [7]:
orbits.orbits

,objId,q,e,inc,Omega,argPeri,tPeri,H,epoch,g,sed_filename,obj_id
0,St500000a,4.91636,0.05966,7.6084,313.711,150.14863,47268.27529,5.63,54800.0,0.15,C.dat,0
1,St500001a,4.96652,0.06370,10.4414,100.539,243.70619,54669.39282,6.25,54800.0,0.15,C.dat,1
2,St500002a,5.01980,0.03319,6.7215,166.404,31.12803,52845.06297,6.36,54800.0,0.15,C.dat,2
3,St500003a,4.75104,0.09715,18.3859,41.238,206.31396,54666.09533,6.61,54800.0,0.15,C.dat,3
4,St500004a,4.91459,0.05269,26.1707,188.834,92.77871,46950.71510,6.92,54800.0,0.15,C.dat,4
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,St5001iza,4.54966,0.12391,25.3756,68.194,213.34991,46774.40386,13.78,54800.0,0.15,C.dat,4995
4996,St5001iAa,4.91108,0.05437,22.6242,180.730,87.73830,46412.55177,13.78,54800.0,0.15,C.dat,4996
4997,St5001iBa,4.79245,0.06708,5.5164,69.568,14.55200,47178.11215,13.78,54800.0,0.15,C.dat,4997
4998,St5001iCa,4.82618,0.08168,9.5511,351.140,266.57704,46114.83788,13.78,54800.0,0.15,C.dat,4998


In [ ]:
# looks like we need to add some columns to this. 


In [10]:
object_observations["magV"]

array([12.90312683, 12.90306915, 12.81952089, ..., 21.45664334,
       21.54504158, 21.54518763], shape=(41574,))

In [11]:
# So, now I want to do a groupby on obj_id, and do lots of calculations on that. 
# Things like, date first detected. Date first characterized, etc. 

In [12]:
object_observations.dtype.names

('obj_id',
 'observationId',
 'sedname',
 'H',
 'time',
 'ra',
 'dec',
 'dradt',
 'ddecdt',
 'phase',
 'solarelon',
 'helio_dist',
 'geo_dist',
 'magV',
 'trueAnomaly',
 'velocity',
 'dmag_color',
 'dmag_trail',
 'dmag_detect',
 'visitExposureTime',
 'fiveSigmaDepth',
 'seeingFwhmGeom',
 'observationStartMJD',
 'night',
 'filter')

In [ ]:
# For some reason we didn't pass along the H absolute mag of the object? Why wouldn't we do that? 
# Is this why we have to specify the orbit file again after we run the first step?
